In [57]:
import mediapipe as mp
import pandas as pd
import json

In [60]:
data_map = pd.read_csv('dataset/ASL_Citizen/splits/train.csv')

In [61]:
with open("dataset/WLASL_v0.3.json", 'r') as file:
    data_map1 = json.load(file)

In [62]:
import re

def remove_numbers_from_string(text):

  return re.sub(r"\d+", "", text)

In [63]:
data_map['Gloss'] = data_map['Gloss'].apply(remove_numbers_from_string).astype('string')

In [64]:
data_map.drop_duplicates(subset=['Gloss'], keep='first', inplace=True)

In [65]:
import cv2

In [66]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [67]:
VIDEOS_DIR = 'dataset/ASL_Citizen/videos'

In [68]:
import os
import json

In [72]:
words = [word["gloss"].upper() for word in data_map1]

In [73]:
selected_words = data_map[data_map["Gloss"].isin(words)]

In [74]:
len(selected_words)

1363

In [76]:
word_coordinates = {}

In [82]:
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    for ind, word in selected_words.iterrows():
        print(ind)
        word_coordinates[word["Gloss"]] = []
        video_path = os.path.join(VIDEOS_DIR, word['Video file'])
        video = cv2.VideoCapture(video_path)
        while video.isOpened():
                ret, frame = video.read()
                if not ret:
                    break

                # Resize the frame to 800x750
                frame = cv2.resize(frame, (800, 750))

                # Convert the frame to RGB
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Detections
                results = hands.process(image_rgb)

                if results.multi_hand_landmarks:
                    # Initialize hand coordinates
                    hand_coordinates = []

                    frame_hands = {
                        "Left Hand Coordinates": [],
                        "Right Hand Coordinates": []
                    }

                        # Process each detected hand
                    for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                        # Get the handedness classification from MediaPipe
                        handedness = results.multi_handedness[idx].classification[0].label
                        hand = hand_landmarks.landmark

                        coordinates = []
                        # Store coordinates and joint index in the hand coordinates list
                        for joint_id, landmark in enumerate(hand):
                            x, y, z = landmark.x, landmark.y, landmark.z
                            joint_data = {
                                joint_id : [x, y, z]
                            }
                            coordinates.append(joint_data)
                        frame_hands[f"{handedness} Hand Coordinates"] = coordinates
                    word_coordinates[word["Gloss"]].append(frame_hands)


0
1
3
4
5
6
8
9
11
14
17
18
19
22
28
29
33
34
37
38
41
42
47
48
49
52
57
60
61
62
64
65
68
70
73
75
77
78
79
83
87
91
93
94
95
97
99
156
187
188
193
196
199
200
202
203
204
205
207
208
209
210
212
214
216
218
219
220
221
222
224
227
228
229
230
231
232
233
234
236
238
239
240
241
242
243
244
245
247
248
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
271
272
274
277
278
279
280
281
282
283
284
285
286
288
289
290
292
296
297
299
300
301
303
304
306
308
310
311
312
314
315
319
322
324
325
326
328
329
331
332
334
335
336
338
339
340
343
344
345
346
348
349
351
352
358
360
361
362
363
365
366
367
368
369
370
371
372
380
382
384
387
389
395
396
400
401
402
403
404
409
411
415
417
418
420
421
422
423
426
428
429
430
432
436
437
441
444
445
447
450
451
452
454
456
458
459
460
461
462
463
467
468
470
474
475
478
479
480
482
483
484
486
487
493
497
580
590
593
597
599
605
607
609
613
615
619
622
623
624
628
631
632
637
638
640
642
646
649
653
659
663
665
668
673
677

KeyboardInterrupt: 

In [83]:
with open('coordinates.json', 'w') as fp:
    json.dump(word_coordinates, fp)

In [84]:
ALPHABET_DIR = "dataset/Alphabets"

In [85]:
import string

In [87]:
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    for alp in string.ascii_uppercase:
        image_path = os.path.join(ALPHABET_DIR, alp+".jpg")
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            # Get the handedness classification from MediaPipe
            handedness = results.multi_handedness[idx].classification[0].label
            hand = hand_landmarks.landmark
            coordinates = []
            # Store coordinates and joint index in the hand coordinates list
            for joint_id, landmark in enumerate(hand):
                x, y, z = landmark.x, landmark.y, landmark.z
                joint_data = {
                    joint_id : [x, y, z]
                }
                coordinates.append(joint_data)
            word_coordinates[alp] = coordinates

In [88]:
with open('coordinates.json', 'w') as fp:
    json.dump(word_coordinates, fp)